In [ ]:
# 高速公路車道檢測
'''
請根據高速公路車道檢測的範例程式來進行修改。
程式首先讀取一張高速公路車道的影像，檢測到左車道與右車道之後，請計算出一條位在左車道與右車道之間的中間線，做為汽車行進調整的依據。
把這條中間線用另一種顏色來標示在高速公路車道影像上，例如原本檢測到的左車道與右車道用藍色，則中間線用黃色。
請計算出影像底部正中間與這條中間線的差距、以及偏左或偏右，標註在影像的左上角。
使用 OpenCV 的 imshow 函式呈現結果，並使用 imwrite 函式輸出成影像檔。
請測試至少三張不同的高速公路車道影像，包括偏左或偏右，以及不同的差距。
此程式只能匯入 cv2 以及 numpy 模組，不可再匯入其他模組。
請把你的 Python 程式檔、輸入影像檔案、輸出影像檔案壓縮起來，繳交你的壓縮檔。

'''
import cv2
import numpy as np

# 讀取圖像
image = cv2.imread("highway.jpg")               # 從指定路徑讀取圖像

# 轉為灰階
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 高斯模糊, 減少邊緣檢測中的噪聲干擾
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Canny 圖像邊緣檢測, 50 和 150 是低閾值和高閾值, 決定那些邊緣被檢測到
edges = cv2.Canny(blurred, 50, 150)

# 定義感興趣區域 (ROI)
height, width = edges.shape                     # 獲取edges圖像的高度和寬度
mask = np.zeros_like(edges)                     # 建立與edges圖像大小相同的全黑遮罩
polygon = np.array([[                           # 定義多邊形作為感興趣區域 ROI
    (int(width * 0.2), height),                 # 左下角點
    (int(width * 0.8), height),                 # 右下角點
    (int(width * 0.7), int(height * 0.5)),      # 右上角點
    (int(width * 0.3), int(height * 0.5))       # 左上角點
]], dtype=np.int32)

# 將多邊形區域充填為白色, 其他區域保持為黑色
cv2.fillPoly(mask, polygon, 255)

# 將遮罩與邊緣圖像進行按位與操作, 保留感興趣區域內的邊緣
roi = cv2.bitwise_and(edges, mask)

# 霍夫直線檢測
lines = cv2.HoughLinesP(
    roi,
    rho=1,                  # 累積器單元的距離解析度(像素)
    theta=np.pi/180,        # 累積器單元的角度解析度(弧度)
    threshold=50,           # 霍夫變換的累積閾值
    minLineLength=80,       # 檢測的最小線段長度(像素)
    maxLineGap=50           # 允許連接的最大間隙(像素)
)
if lines is None:
    print("未檢測到任何直線")   # 如果未檢測到任何直線
else:
    print(f"檢測到的直線數量:{len(lines)}") # 輸出檢測到的直線數量

# 繪製檢測到的直線
output = np.copy(image)                    # 複製原始圖像, 用於繪製直線
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]            # 取得每條直線的兩個端點座標
        # 繪製藍色直線, 線寬為5
        cv2.line(output, (x1, y1), (x2, y2), (255, 0, 0), 5)

# 顯示結果
cv2.imshow('Original Image', image)         # 顯示原始圖像
cv2.imshow('Canny Edges', edges)            # 顯示邊緣檢測結果
cv2.imshow('Region of Interest', roi)       # 顯示應用了 ROI的邊緣圖像
cv2.imshow('Detected Lanes', output)        # 顯示檢測到車道線後的最終結果

cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np

def detect_lanes_and_center_line(image_path):
    """
    對高速公路車道影像進行檢測，找出左右車道線、中間線，並計算偏離程度。
    
    Args:
        image_path (str): 輸入影像檔案的路徑。
    """

    # 讀取影像
    image = cv2.imread(image_path)
    if image is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 1. 影像預處理
    height, width = image.shape[:2]
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # 2. 定義感興趣區域 (ROI)
    mask = np.zeros_like(edges)
    polygon = np.array([
        [(int(width * 0.2), height), (int(width * 0.8), height), 
         (int(width * 0.7), int(height * 0.5)), (int(width * 0.3), int(height * 0.5))]
    ], dtype=np.int32)
    cv2.fillPoly(mask, polygon, 255)
    roi = cv2.bitwise_and(edges, mask)

    # 3. 霍夫直線檢測
    lines = cv2.HoughLinesP(
        roi, rho=1, theta=np.pi/180, threshold=50, 
        minLineLength=80, maxLineGap=50
    )

    left_lanes = []
    right_lanes = []
    output = np.copy(image)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if x1 == x2:
                continue
            
            slope = (y2 - y1) / (x2 - x1)
            
            # 根據斜率將線分類為左車道或右車道
            # 左車道線斜率通常為負，右車道線斜率通常為正
            if -0.8 < slope < -0.2:
                left_lanes.append(line)
                cv2.line(output, (x1, y1), (x2, y2), (255, 0, 0), 5) # 繪製左車道 (藍色)
            elif 0.2 < slope < 0.8:
                right_lanes.append(line)
                cv2.line(output, (x1, y1), (x2, y2), (255, 0, 0), 5) # 繪製右車道 (藍色)

    # 4. 計算中間線
    left_x_coords = []
    right_x_coords = []
    
    # 提取所有左車道和右車道線的 x 座標
    if left_lanes:
        for line in left_lanes:
            left_x_coords.extend([line[0][0], line[0][2]])
    if right_lanes:
        for line in right_lanes:
            right_x_coords.extend([line[0][0], line[0][2]])

    # 確保有檢測到左右車道線
    if not left_x_coords or not right_x_coords:
        cv2.putText(output, "Lane not detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Final Result', output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return

    # 計算底部和頂部的中間 x 座標
    bottom_center_x = (np.mean(left_x_coords) + np.mean(right_x_coords)) / 2
    top_center_x = (int(width * 0.3) + int(width * 0.7)) / 2 # ROI 的頂部中點

    # 5. 繪製中間線 (黃色)
    cv2.line(output, (int(bottom_center_x), height), (int(top_center_x), int(height * 0.5)), (0, 255, 255), 5)

    # 6. 計算與影像中心點的差距
    image_center_x = width / 2
    deviation = bottom_center_x - image_center_x
    
    # 判斷偏左或偏右
    if deviation > 0:
        direction = "Right"
    elif deviation < 0:
        direction = "Left"
    else:
        direction = "Center"

    # 7. 標註資訊在影像上
    text = f"Deviation: {abs(deviation):.2f} pixels ({direction})"
    cv2.putText(output, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # 8. 顯示和輸出結果
    cv2.imshow('Final Result', output)
    cv2.imwrite(f"{image_path.split('.')[0]}_output.jpg", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 測試不同的影像
# 請準備三張不同的高速公路車道影像，例如：
# 1. 影像正中對準車道
# 2. 影像偏左
# 3. 影像偏右
# 將它們命名為 "highway_center.jpg", "highway_left.jpg", "highway_right.jpg"
detect_lanes_and_center_line("highway.jpg")
# detect_lanes_and_center_line("highway_left.jpg")
# detect_lanes_and_center_line("highway_right.jpg")